In [27]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [28]:
num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [29]:
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X.sum(0, keepdim=True), X.sum(1, keepdim=True)

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

In [30]:
def softmax(X):
    X_exp = torch.exp(X)  # 对X的每一个元素求e
    partition = X_exp.sum(1, keepdim=True)  # 分母是所有元素求e的和
    return X_exp / partition

In [31]:
# 测试softmax函数的正确性
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(tensor([[0.2383, 0.1437, 0.1517, 0.2968, 0.1694],
         [0.1852, 0.1339, 0.2712, 0.2979, 0.1117]]),
 tensor([1.0000, 1.0000]))

In [32]:
def net(X):
    """实现softmax回归模型"""
    arr = torch.matmul(X.reshape((-1, W.shape[0])), W) + b  # y = XW + b
    return softmax(arr)

In [33]:
# 创建一个数据y_hat，其中包含2个样本在3个类别的预测概率，使用y作为y_hat中概率的索引
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], 
                      [0.3, 0.2, 0.5]])
y_hat[[0, 1], y] # 这里表示y_hat[[0, 1], [0, 2]], 即取出y_hat[0, 0]和y_hat[1, 2]


tensor([0.1000, 0.5000])

In [34]:
def cross_entropy(y_hat, y):
    """实现交叉熵损失函数"""
    return -torch.log(y_hat[range(len(y_hat)), y])

cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

In [35]:
def accuracy(y_hat, y):
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)  # 求出每一行的最大值对应的下标
    cmp = y_hat.type(y.dtype) == y # 比较新y_hat数组和y数组，求出有多少个元素是相等的
    return float(cmp.type(y.dtype).sum())

accuracy(y_hat, y) / len(y)

0.5

In [52]:
class Accumulator:
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n
    
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)] # 对self.data里的每个数加上args中的每个数（args是传进来的参数）

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [53]:
def evaluate_accuracy(net, data_iter):
    """计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module): # isinstance：判断两个类型是否相同
        net.eval() # 将模型设置为评估模式
    metric = Accumulator(2) # 创建一个长度为2的累加器
    for X, y in data_iter:
        metric.add(accuracy(net(X), y), y.numel()) # 当前这一批(正确分类的元素个数，总元素个数)
    return metric[0] / metric[1]

evaluate_accuracy(net, test_iter)

0.0508

In [54]:
def train_epoch_ch3(net, train_iter, loss, updater):
    """训练模型一个迭代周期"""
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用Pytorch内置的优化器和损失函数
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            # 使用定制的优化器和损失函数
            l.sum().backward()
            updater(X.shape[0])
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    # 返回训练损失和训练精度
    return metric[0] / metric[2], metric[1] / metric[2]

